In [22]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import re

In [23]:
# Specify the directory containing CSV files
csv_directory_path = r"E:\Code\data_blog\blog_files\data"

# List all files in the directory
all_files = os.listdir(csv_directory_path)
specific_csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list of full file paths for the specific CSV files
specific_csv_paths = [os.path.join(csv_directory_path, f) for f in specific_csv_files]

In [24]:
from collections import defaultdict

In [25]:
# Initialize the dictionary with default lists
file_categories = defaultdict(list)

# Categorize filenames
for filename in specific_csv_files:
    if 'biometrics' in filename.lower():
        file_categories['biometrics'].append(os.path.join(csv_directory_path, filename))
    elif 'dailysummary' in filename.lower():
        file_categories['dailysummary'].append(os.path.join(csv_directory_path, filename))
    elif 'servings' in filename.lower():
        file_categories['servings'].append(os.path.join(csv_directory_path, filename))

In [26]:
file_categories

defaultdict(list,
            {'biometrics': ['E:\\Code\\data_blog\\blog_files\\data\\biometrics.csv'],
             'dailysummary': ['E:\\Code\\data_blog\\blog_files\\data\\dailysummary.csv'],
             'servings': ['E:\\Code\\data_blog\\blog_files\\data\\servings.csv']})

In [27]:
specific_csv_paths

['E:\\Code\\data_blog\\blog_files\\data\\biometrics.csv',
 'E:\\Code\\data_blog\\blog_files\\data\\dailysummary.csv',
 'E:\\Code\\data_blog\\blog_files\\data\\servings.csv']

In [28]:
df = pd.read_csv(specific_csv_paths[2])

In [29]:
df.columns

Index(['Day', 'Group', 'Food Name', 'Amount', 'Energy (kcal)', 'Alcohol (g)',
       'Caffeine (mg)', 'Water (g)', 'B1 (Thiamine) (mg)',
       'B2 (Riboflavin) (mg)', 'B3 (Niacin) (mg)',
       'B5 (Pantothenic Acid) (mg)', 'B6 (Pyridoxine) (mg)',
       'B12 (Cobalamin) (µg)', 'Folate (µg)', 'Vitamin A (µg)',
       'Vitamin C (mg)', 'Vitamin D (IU)', 'Vitamin E (mg)', 'Vitamin K (µg)',
       'Calcium (mg)', 'Copper (mg)', 'Iron (mg)', 'Magnesium (mg)',
       'Manganese (mg)', 'Phosphorus (mg)', 'Potassium (mg)', 'Selenium (µg)',
       'Sodium (mg)', 'Zinc (mg)', 'Carbs (g)', 'Fiber (g)', 'Starch (g)',
       'Sugars (g)', 'Added Sugars (g)', 'Net Carbs (g)', 'Fat (g)',
       'Cholesterol (mg)', 'Monounsaturated (g)', 'Polyunsaturated (g)',
       'Saturated (g)', 'Trans-Fats (g)', 'Omega-3 (g)', 'Omega-6 (g)',
       'Cystine (g)', 'Histidine (g)', 'Isoleucine (g)', 'Leucine (g)',
       'Lysine (g)', 'Methionine (g)', 'Phenylalanine (g)', 'Protein (g)',
       'Threonine (g)', 

In [30]:
# Function to transform a single column name
def transform_column_name(col_name):
    col_name = col_name.lower()
    col_name = re.sub(r' ', '_', col_name)
    col_name = re.sub(r'[^a-z0-9_]', '', col_name)
    if not re.match(r'^[a-z_]', col_name):
        col_name = f'_{col_name}'
    col_name = col_name[:128]
    return col_name

new_column_names = [transform_column_name(col) for col in df.columns]
df.columns = new_column_names

In [31]:
df.columns

Index(['day', 'group', 'food_name', 'amount', 'energy_kcal', 'alcohol_g',
       'caffeine_mg', 'water_g', 'b1_thiamine_mg', 'b2_riboflavin_mg',
       'b3_niacin_mg', 'b5_pantothenic_acid_mg', 'b6_pyridoxine_mg',
       'b12_cobalamin_g', 'folate_g', 'vitamin_a_g', 'vitamin_c_mg',
       'vitamin_d_iu', 'vitamin_e_mg', 'vitamin_k_g', 'calcium_mg',
       'copper_mg', 'iron_mg', 'magnesium_mg', 'manganese_mg', 'phosphorus_mg',
       'potassium_mg', 'selenium_g', 'sodium_mg', 'zinc_mg', 'carbs_g',
       'fiber_g', 'starch_g', 'sugars_g', 'added_sugars_g', 'net_carbs_g',
       'fat_g', 'cholesterol_mg', 'monounsaturated_g', 'polyunsaturated_g',
       'saturated_g', 'transfats_g', 'omega3_g', 'omega6_g', 'cystine_g',
       'histidine_g', 'isoleucine_g', 'leucine_g', 'lysine_g', 'methionine_g',
       'phenylalanine_g', 'protein_g', 'threonine_g', 'tryptophan_g',
       'tyrosine_g', 'valine_g', 'category'],
      dtype='object')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675 entries, 0 to 674
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   day                     675 non-null    object 
 1   group                   675 non-null    object 
 2   food_name               675 non-null    object 
 3   amount                  675 non-null    object 
 4   energy_kcal             653 non-null    float64
 5   alcohol_g               373 non-null    float64
 6   caffeine_mg             420 non-null    float64
 7   water_g                 465 non-null    float64
 8   b1_thiamine_mg          456 non-null    float64
 9   b2_riboflavin_mg        452 non-null    float64
 10  b3_niacin_mg            452 non-null    float64
 11  b5_pantothenic_acid_mg  401 non-null    float64
 12  b6_pyridoxine_mg        421 non-null    float64
 13  b12_cobalamin_g         394 non-null    float64
 14  folate_g                432 non-null    fl

In [33]:
import pandas as pd
import psycopg2
from psycopg2 import sql, connect, OperationalError, errorcodes, errors
import sys
class DatabaseManager:
    """
    A class to manage database connections using psycopg2.

    ...

    Attributes
    ----------
    conn : psycopg2 connection object
        connection to the database to handle database actions

    Methods
    -------
    create_tables():
        Creates the `parameters` and `blendermann_coefficients` tables for the database.

    create_entry(table_name, fields, values):
        Creates an entry in the table given the values and fields to insert them into.
    
    update_value(table_name, fields, entry):
        Updates a specified entry in the database given the fields and entry details.
    
    get_table(table_name):
        Obtains all the entries of a table when given the table name.

    get_value(table_name, fields, values):
        Runs a select statement, filtering the values by supplying the WHERE clause of the SQL query using the provided fields and values.
    
    delete_value(table_name, fields, values):
        Deletes the entries of a table in the database given the fields and their corresponding values.

    get_fields(table_name):
        Returns the fields of the database table given the table name.

    get_table_desc(table_name):
        Returns a tuple of the fields as well as their type codes.
    
    insert_dataframe(table_name, df):
        Inserts a pandas dataframe as a series of INSERT statements into the database table table_name
    """

    def __init__(self, user, dbname, password, host, port):
        """
        Constructs a database connection using the provided parameters.
        Configured in the provided .env file, can be manually set as well by calling an instance of the manager as DatabaseManager(user, dbname, password, host, port)

        ...

        Parameters
        ----------
            user : str
                username credentials of the database
            dbname : str
                name of the database to request access to
            password : str
                password credentials of the database
            host : str
                host address of the database
            port : str
                port address of the database
        
        Returns
        ----------
        True: if the database was connected
        False: if the database was not connected
        """
        connection_string = f'user={user} dbname={dbname} password={password} host={host} port={port}'

        try:
            self.conn = psycopg2.connect(connection_string)
            print('Database manager has connected to ' + str(dbname)) # Logging to stdout
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn = None

    def create_entry(self, table_name, fields, values):
        """
        Creates an entry in the database

        ...

        Parameters
        ----------
        table_name: str
            database table to insert the entry into
        fields: tuple
            list of fields to insert the entry into
        values: tuple
            list of values per specfied field of the entry
        
        Returns
        ----------
        True: if the entry was created
        False: if the entry was not created
        """

        cur = self.conn.cursor()
        
        # check if entry is present before inserting
        column_names, rows = self.get_value(table_name, fields, values)
        if rows == None or len(rows) >= 1:
            print('Entry already exists. Insertion is not continued.')
            return False

        # query for inserting values
        query = sql.SQL("""
        INSERT INTO {table} ({fields})
        VALUES ({values})
        """).format(
            table=sql.Identifier(table_name),
            fields=sql.SQL(', ').join(map(sql.Identifier, fields)),
            values=sql.SQL(', ').join(sql.Placeholder() * len(values))
        )

        try:
            cur.execute(query, values)
            self.conn.commit()
            print('Entry insertion of `ship_id:' + values[0] + '` complete') # Logging to stdout
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn.rollback()
            return False
        cur.close()
        return True

    def update_value(self, table_name, fields, entry):
        """
        Updates an entry given a primary key.

        ...

        Parameters
        ----------
        table_name: str
            database table that contains the entry
        fields: tuple
            list of fields to update the entry to
            first element is assumed to be the primary key
        values: tuple
            list of values to update the entry to
            first element is assumed to be the primary key
        
        Returns
        ----------
        True: if the entry was updated
        False: if the entry was not updated
        """

        cur = self.conn.cursor()
        pk_value = entry[0]
        new_values = entry[1:]

        # query for updating values from table
        set_clause = sql.SQL(', ').join(
            sql.SQL("{field} = %s").format(field=sql.Identifier(field))
            for field in fields[1:]
        )
        where_clause = sql.SQL("{pk_field} = %s").format(pk_field=sql.Identifier(fields[0]))

        query = sql.SQL("""
        UPDATE {table}
        SET {set_clause}
        WHERE {where_clause}
        """).format(
            table=sql.Identifier(table_name),
            set_clause=set_clause,
            where_clause=where_clause
        )

        values = new_values + [pk_value]

        try:
            cur.execute(query, values)
            self.conn.commit()
            print(str(pk_value) + ' updated to new values: ' + str(entry) + '.') # Logging to stdout
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn.rollback()
            return False
        cur.close()
        return True

    def get_table(self, table_name):
        """
        Returns the rows of a table as a mutable object.

        ...

        Parameters
        ----------
        table_name: str
            name of the database table to retrieve entries from
        
        Returns
        ----------
        column_names: list or None
            list is returned if no error occurred, None otherwise
        rows: list or None
            list is returned if no error occurred, None otherwise
        """
        cur = self.conn.cursor()

        # query for reading from table
        query = sql.SQL("""
        SELECT * FROM {table}
        """).format(
            table=sql.Identifier(table_name)
        )
        try:
            cur.execute(query, table_name)
            rows = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print('Fetched ' + str(len(rows)) + ' entries from ' + table_name) # Logging to stdout
            cur.close()
            return column_names, rows
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn.rollback()
            return None, None
        
    def get_value(self, table_name, fields, values):
        """
        Gets values from the database table given a set of fields and values to search from
        Note that the template for obtaining the value only works for fields that have one value, i.e. primary key fields such as `ship_id` or `vessel_type`
        The template for obtaining values within an array or timestamp will vary and need to be created in a separate function if needed
        ...

        Parameters
        ----------
        table_name: str
            name of the database table to retrieve entries from
        fields: tuple
            list of fields to find entry
        values: tuple
            list of values to find entry
        Returns
        ----------
        column_name, rows: list or None
            returns mutable objects for both variables if elements that follow the conditions were found, None otherwise
        """

        cur = self.conn.cursor()

        # query for selecting with set values
        where_clause = sql.SQL(' AND ').join(
            sql.SQL("{field} = %s").format(field=sql.Identifier(field))
            for field in fields
        )
        query = sql.SQL("""
        SELECT * FROM {table}
        WHERE {where_clause}
        """).format(
            table=sql.Identifier(table_name),
            where_clause=where_clause
        )
        
        try:
            cur.execute(query, values)
            rows = cur.fetchall()
            column_names = [desc[0] for desc in cur.description]
            print('Fetched ' + str(len(rows)) + ' entries from ' + table_name) # Logging to stdout
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn.rollback()
            return None, None
        cur.close()
        return column_names, rows

    def delete_value(self, table_name, fields, values):
        """
        Deletes values from the database table

        ...

        Parameters
        ----------
        table_name: str
            name of the database table to retrieve entries from
        fields: tuple
            list of fields to find entry to delete
        values: tuple
            list of values to find entry to delete
        Returns
        ----------
        True: if the entry was deleted
        False: if the entry was not deleted
        """

        cur = self.conn.cursor()
        
        # query for deleting with conditions
        where_clause = sql.SQL(' AND ').join(
            sql.SQL("{field} = %s").format(field=sql.Identifier(field))
            for field in fields
        )
        query = sql.SQL("""
        DELETE FROM {table}
        WHERE {where_clause}
        """).format(
            table=sql.Identifier(table_name),
            where_clause=where_clause
        )
        
        try:
            cur.execute(query, values)
            self.conn.commit()
            print('Deleted ' + str(values) + ' from ' + str(table_name)) # Logging to stdout
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn.rollback()
            return False
        cur.close()
        return True

    def get_fields(self, table_name):
        """
        Gets the list of fields of the database table

        ...

        Parameters
        ----------
        table_name: str
            name of the database table to get fields from

        Returns
        ----------
        column_names: list or None
            returns a mutable object of the list of table fields or None if an error occurred
        """
        cur = self.conn.cursor()
        # Execute a simple SELECT query
        try:
            cur.execute(sql.SQL("SELECT * FROM {table} LIMIT 0").format(
                table=sql.Identifier(table_name)
            ))
            column_names = [desc[0] for desc in cur.description]
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn.rollback()
            return None
        cur.close()
        return column_names

    def get_table_desc(self, table_name):
        """
        Gets the description of the datbaase table as a mutable object of columns where the first element is the name of the column and the second is the type code

        ...

        Parameters
        ----------
        table_name: str
            name of the database table to get fields from

        Returns
        ----------
        descriptions: list or None
            returns a mutable object of the description of the table's fields or None if an error occurred
        """
        cur = self.conn.cursor()
        # Execute a simple SELECT query
        try:
            cur.execute(sql.SQL("SELECT * FROM {table} LIMIT 0").format(
                table=sql.Identifier(table_name)
            ))
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn.rollback()
            return None
        cur.close()
        return cur.description

    def insert_dataframe(self, table_name, df):
        """
        Inserts a pandas dataframe as a series of INSERT statements into specified database table
        Used when extracting data from a CSV file
        Could also be used for general purposes such as having a pandas dataframe from any source
        ...

        Parameters
        ----------
        table_name: str
            name of the database table to insert entries into
        df: dataframe
            pandas dataframe to insert all entries from

        Returns
        ----------
        True: if the dataframe was inserted
        False: if the dataframe was not inserted
        """

        cur = self.conn.cursor()
        columns = list(df.columns)
        values = [tuple(row) for row in df.itertuples(index=False, name=None)]
        print('Inserting ' + str(len(values)) + ' to ' + table_name) # Logging to stdout
        for value in values:
            self.create_entry(table_name, tuple(columns), tuple(value))
        print('Dataframe insertion to ' + table_name + ' complete.') # Logging to stdout
        cur.close()

    def execute_query(self, query):
        cur = self.conn.cursor()
        try:
            cur.execute(query)
            print("query executed")
            self.conn.commit()
        except Exception as err:
            self.print_psycopg2_exception(err)
            self.conn.rollback()
            return False
            
        cur.close()
        return True

    def print_psycopg2_exception(self, err):
        """
        Prints the error obtained from handling database functions with psycopg2
        ...

        Parameters
        ----------
        err: Exception
            error to obtain details from

        Returns
        ----------
        None
        """

        err_type, err_obj, traceback = sys.exc_info()
        line_num = traceback.tb_lineno
        print ("\npsycopg2 ERROR:", err, "on line number:", line_num)
        print ("psycopg2 traceback:", traceback, "-- type:", err_type)
        print ("\nextensions.Diagnostics:", err.diag)
        print ("pgerror:", err.pgerror)
        print ("pgcode:", err.pgcode, "\n")

In [34]:
load_dotenv()
dbManager = DatabaseManager(os.getenv('user'), os.getenv('dbname'), os.getenv('password'), os.getenv('host'), os.getenv('port'))

Database manager has connected to health_tracking


Servings

In [35]:
query = """
CREATE TABLE IF NOT EXISTS servings (
    day                     TEXT NOT NULL,
    \"group\"                   TEXT,
    food_name               TEXT NOT NULL,
    amount                  TEXT NOT NULL,
    energy_kcal             FLOAT,
    alcohol_g               FLOAT,
    caffeine_mg             FLOAT,
    water_g                 FLOAT,
    b1_thiamine_mg          FLOAT,
    b2_riboflavin_mg        FLOAT,
    b3_niacin_mg            FLOAT,
    b5_pantothenic_acid_mg  FLOAT,
    b6_pyridoxine_mg        FLOAT,
    b12_cobalamin_g         FLOAT,
    folate_g                FLOAT,
    vitamin_a_g             FLOAT,
    vitamin_c_mg            FLOAT,
    vitamin_d_iu            FLOAT,
    vitamin_e_mg            FLOAT,
    vitamin_k_g             FLOAT,
    calcium_mg              FLOAT,
    copper_mg               FLOAT,
    iron_mg                 FLOAT,
    magnesium_mg            FLOAT,
    manganese_mg            FLOAT,
    phosphorus_mg           FLOAT,
    potassium_mg            FLOAT,
    selenium_g              FLOAT,
    sodium_mg               FLOAT,
    zinc_mg                 FLOAT,
    carbs_g                 FLOAT,
    fiber_g                 FLOAT,
    starch_g                FLOAT,
    sugars_g                FLOAT,
    added_sugars_g          FLOAT,
    net_carbs_g             FLOAT,
    fat_g                   FLOAT,
    cholesterol_mg          FLOAT,
    monounsaturated_g       FLOAT,
    polyunsaturated_g       FLOAT,
    saturated_g             FLOAT,
    transfats_g             FLOAT,
    omega3_g                FLOAT,
    omega6_g                FLOAT,
    cystine_g               FLOAT,
    histidine_g             FLOAT,
    isoleucine_g            FLOAT,
    leucine_g               FLOAT,
    lysine_g                FLOAT,
    methionine_g            FLOAT,
    phenylalanine_g         FLOAT,
    protein_g               FLOAT,
    threonine_g             FLOAT,
    tryptophan_g            FLOAT,
    tyrosine_g              FLOAT,
    valine_g                FLOAT,
    category                TEXT
)
"""
dbManager.execute_query(query)

query executed


True

Biometrics

In [36]:
query = """
CREATE TABLE IF NOT EXISTS biometrics (
    day     TEXT NOT NULL,
    "group" TEXT NOT NULL,
    metric  TEXT NOT NULL,
    unit    TEXT NOT NULL,
    amount  FLOAT NOT NULL
)
"""
dbManager.execute_query(query)

query executed


True

Daily Summary

In [37]:
query = """
CREATE TABLE dailysummary (
    date                        TEXT NOT NULL,
    energy_kcal                 FLOAT NOT NULL,
    alcohol_g                   FLOAT,
    caffeine_mg                 FLOAT,
    water_g                     FLOAT,
    b1_thiamine_mg              FLOAT,
    b2_riboflavin_mg            FLOAT,
    b3_niacin_mg                FLOAT,
    b5_pantothenic_acid_mg      FLOAT,
    b6_pyridoxine_mg            FLOAT,
    b12_cobalamin_g            FLOAT,
    folate_g                   FLOAT,
    vitamin_a_g                FLOAT,
    vitamin_c_mg                FLOAT,
    vitamin_d_iu                FLOAT,
    vitamin_e_mg                FLOAT,
    vitamin_k_g                FLOAT,
    calcium_mg                  FLOAT,
    copper_mg                   FLOAT,
    iron_mg                     FLOAT,
    magnesium_mg                FLOAT,
    manganese_mg                FLOAT,
    phosphorus_mg               FLOAT,
    potassium_mg                FLOAT,
    selenium_g                 FLOAT,
    sodium_mg                   FLOAT,
    zinc_mg                     FLOAT,
    carbs_g                     FLOAT,
    fiber_g                     FLOAT,
    starch_g                    FLOAT,
    sugars_g                    FLOAT,
    added_sugars_g              FLOAT,
    net_carbs_g                 FLOAT,
    fat_g                       FLOAT,
    cholesterol_mg              FLOAT,
    monounsaturated_g           FLOAT,
    polyunsaturated_g           FLOAT,
    saturated_g                 FLOAT,
    transfats_g                FLOAT,
    omega3_g                    FLOAT,
    omega6_g                    FLOAT,
    cystine_g                   FLOAT,
    histidine_g                 FLOAT,
    isoleucine_g                FLOAT,
    leucine_g                   FLOAT,
    lysine_g                    FLOAT,
    methionine_g                FLOAT,
    phenylalanine_g             FLOAT,
    protein_g                   FLOAT,
    threonine_g                 FLOAT,
    tryptophan_g                FLOAT,
    tyrosine_g                  FLOAT,
    valine_g                    FLOAT,
    completed                   BOOLEAN
)
"""
dbManager.execute_query(query)


psycopg2 ERROR: relation "dailysummary" already exists
 on line number: 410
psycopg2 traceback: <traceback object at 0x0000026D717C9540> -- type: <class 'psycopg2.errors.DuplicateTable'>

extensions.Diagnostics: <psycopg2.extensions.Diagnostics object at 0x0000026D71EECCD0>
pgerror: ERROR:  relation "dailysummary" already exists

pgcode: 42P07 



False

In [38]:
for key, values in file_categories.items():
    for value in values:
        df = pd.read_csv(value)
        if 'category' in df.columns:
            df = df['category'].fillna("")
        df = df.fillna(0)
        new_column_names = [transform_column_name(col) for col in df.columns]
        df.columns = new_column_names
        dbManager.insert_dataframe(table_name=key, df=df)

Inserting 32 to biometrics
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already exists. Insertion is not continued.
Fetched 7 entries from biometrics
Entry already e

In [39]:
query = """
DROP TABLE dailysummary
"""
dbManager.execute_query(query)

query executed


True

In [40]:
query = """
DROP TABLE servings
"""
dbManager.execute_query(query)

query executed


True

In [41]:
dbManager.get_table("servings")


psycopg2 ERROR: relation "servings" does not exist
LINE 2:         SELECT * FROM "servings"
                              ^
 on line number: 214
psycopg2 traceback: <traceback object at 0x0000026D71FA1700> -- type: <class 'psycopg2.errors.UndefinedTable'>

extensions.Diagnostics: <psycopg2.extensions.Diagnostics object at 0x0000026D71D34A50>
pgerror: ERROR:  relation "servings" does not exist
LINE 2:         SELECT * FROM "servings"
                              ^

pgcode: 42P01 



(None, None)

In [42]:
dbManager.get_value()

TypeError: DatabaseManager.get_value() missing 3 required positional arguments: 'table_name', 'fields', and 'values'